# Data Retrieval
Notebook that retrieves all the data needed. Doesn't compute similarity scores (see other notebooks).

In [1]:
import requests
from tqdm import tqdm
import json
from PIL import Image, ImageDraw
import io
import os

In [2]:
data_path = "../data"
client_data_path = os.path.join(data_path, "client_data")
pokemon_data_path = os.path.join(data_path, "pokemon_data.json")

In [3]:
def save_json(object_, path):
    # Serializing json
    json_object = json.dumps(object_, indent=4)
    
    # Writing to sample.json
    with open(path, "w") as outfile:
        outfile.write(json_object)

In [4]:
def load_json(path):
    # Opening JSON file
    with open(path, 'r') as openfile:
        # Reading from json file
        json_object = json.load(openfile)
        return json_object

## All pokemon data retrieval
Queries the PokeAPI for Pokemon data and saves this. Should only be run once unless modified.

In [ ]:
url = "https://pokeapi.co/api/v2/pokemon/"
all_pokemon_data = {}
while url:
    # Query the Pokemon API for information about this page of Pokemon
    response = requests.get(url)
    pokemon_list = response.json()
    # Loop through the list of Pokemon
    for pokemon in pokemon_list["results"]:
        
        # Query the Pokemon API for information about this specific Pokemon
        response = requests.get(pokemon["url"])
        pokemon_data = response.json()
        
        
        pid = pokemon_data["id"]
        all_pokemon_data[pid] = pokemon_data
        
        # Query the Pokemon API for the sprite of this Pokemon
        sprite_url = pokemon_data["sprites"]["front_default"]
        sprite_response = requests.get(sprite_url)

        # Save the sprite to a file
        with open("../data/sprites/%d.png"%pid, "wb") as f:
            f.write(sprite_response.content)
        
        # Update the URL to query the next page of results, if there is one
        url = pokemon_list["next"]
        
    # print(url)

In [24]:
# Get Pokedex entries
for pid in tqdm(all_pokemon_data):
    pokemon_data = all_pokemon_data[pid]
    # Query the Pokemon API for the pokedex entry of this Pokemon
    pokedex_response = requests.get(pokemon_data["species"]["url"])
    pokedex_data = pokedex_response.json()
    pokedex_entry = next(entry for entry in pokedex_data["flavor_text_entries"][::-1] if entry["language"]["name"] == "en")
    all_pokemon_data[pid]["flavor_text"] = pokedex_entry["flavor_text"]

100%|██████████| 899/899 [01:06<00:00, 13.60it/s]


In [27]:
save_json(all_pokemon_data, pokemon_data_path)

## Data manipulation
Filters and modifies pokemon data that will be served on the frontend.

In [5]:
all_pokemon_data = load_json(pokemon_data_path)

In [9]:
# filter pokemon data JSON
filtered_data = {}
all_pokemon_names = []
for pid in all_pokemon_data:
    pokemon_data = all_pokemon_data[pid]
    all_pokemon_names.append(pokemon_data["species"]["name"].capitalize())

for pid in tqdm(all_pokemon_data):
    pokemon_data = all_pokemon_data[pid]
    filtered_pokemon_data = {
        "name": pokemon_data["species"]["name"].capitalize(),
    }
    
    pokedex_entry = pokemon_data["flavor_text"]
    replacement_token = "[REDACTED]"
    for name in all_pokemon_names:
        pokedex_entry = pokedex_entry.replace(name, replacement_token)
    filtered_pokemon_data["flavor_text"] = pokedex_entry
        
    filtered_data[pid] = filtered_pokemon_data
    
save_json(filtered_data, os.path.join(client_data_path, "pokemon_data.json"))

100%|██████████████████████████████████████████████████████████████████████| 899/899 [00:00<00:00, 4841.24it/s]


In [50]:
# create silhouettes
for pid in tqdm(all_pokemon_data):
    img = Image.open("../data/sprites/%d.png"%pid).convert("RGBA")
    new_img = Image.new("RGBA", img.size, (0, 0, 0, 0))

    # Draw the sprite onto the black
    for x in range(img.width):
        for y in range(img.height):
            # If the pixel is not transparent,
            if img.getpixel((x, y))[3] != 0:
                # Set the corresponding pixel in the new image to black
                new_img.putpixel((x, y), (0, 0, 0, 255))

    # Save the silhouette image to a file
    new_img.save("../data/client_data/mystery_sprites/%d.png"%pid)

 99%|█████████▉| 891/899 [00:09<00:00, 98.83it/s] 


FileNotFoundError: [Errno 2] No such file or directory: '../data/sprites/899.png'